# Imports sous Notebook Python
Dans ce TP, vous allez essentiellement programmer des classes qui vous permettront de construire des réseaux de neurones. Ces classes seront enregistrées dans un fichier `Neural.py` qui contient déjà la classe `Generic`. Cependant, le comportement par défaut d'un Notebook quand on demande d'importer un fichier est de ne pas le relire !!! Ainsi vos modifications dans le fichier `Neural.py` ne seront pas prises en compte. Pour que ce soit le cas, il faut lancer les commandes suivantes :

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import Neural as Neur

In [2]:
import Neural as Neur
L=Neur.Generic()
print(L.backward(1))
# Vous devez trouver
# (None, None)

(None, None)


Copiez-collez la classe `Generic` en une classe `Arctan` et essayez de voir si python recharge bien le fichier Neural quand vous lancez la commande `import`



In [4]:
import Neural as Neur
L=Neur.Arctan()
X=1
print(L.backward(1))

# Vous devez trouver
# (None, None)

TypeError: __init__() missing 2 required positional arguments: 'nb_entree' and 'nb_sortie'

# Définition d'une couche
Le fichier `Neural.py` sera une librairie qui nous permettra de construire nos réseaux de neurones, nous allons au fur et à mesure construire nos différent types de couche.
Toute nos couches auront par défaut au moins les mêmes fonctions et variables que la couche `Generic`, et elles peuvent en avoir plus. Mathématiquement, on rappelle que une couche est une fonction $F$ qui prend des données $X$ et qui rend un vecteur $Y$. La couche a en interne des `paramètres` qui ont vocation à êtres `appris`. Les paramètres sont notés $\theta$. On a ainsi
$$ Y=F(\theta,X)$$

Les fonctions sont :

* `set_params` : sert à fixer $\theta$, les paramètres internes de la couche.
* `get_params` : sert à récupérer $\theta$, les paramètres internes de la couche.
* `forward` : Applique la fonction $F$ aux données $X$ et aux paramètres internes $\theta$ et rend $Y$.
* `backward` : Applique la rétro-propagation du gradient à un gradient de sortie `grad_sortie` et calcule 
  * le gradient local `grad_local` qui est le gradient par rapport aux paramètres internes
  * le gradient d'entrée `grad_entree` qui est le gradient par rapport à la variable $X$

Les variables internes sont :
* `self.nb_params` : la taille du vecteur $\theta$.

On doit se souvenir d'un truc concernant les tailles de vecteur.
* Le vecteur $X$ et `grad_entree` sont de même taille (`grad_entree` est le gradient par rapport à $X$).
* Le vecteur $Y$ et `grad_sortie` sont de même taille (`grad_sortie` est le gradient par rapport à $Y$).
* Le vecteur $\theta$ et `grad_local` sont de même taille, qui est `self.nb_params` (`grad_local` est le gradient par rapport à $\theta$).


On remarque que les couches sauvent automatiquement la variable `X` qui leur est donnée dans la fonction `forward`. Cette variable est stockée dans `self.save_X`. Il est très utile de sauver la variable `X` car elle est réutilisée dans la fonction `backward`. Cependant, cela va rendre notre réseau de neurones très gourmand en mémoire. D'autres choix peuvent être faits, mais nous nous fixerons sur ces choix là pendant les TPs.

# Structure des données

Les données $X$, $Y$ et les paramètres $\theta$ sont des tableaux n


# Implémentation de la couche Arctan
Passons maintenant à notre première couche, vous avez normalement copié-collé la classe `Generic` en une classe `Arctan`. Nous allons remplir cette classe.

La couche `Arctan` est une couche qui prend comme vecteur d'entrée $X$ de taille $p$ et qui rend un vecteur $Y$ de taille $p$ tel que
$$Y[i]=\phi(X[i]) \quad \forall 1\le i\le p$$

Où $\phi$ est la fonction arctangente. Cette couche n'a pas de paramètres locaux (`self.nb_params=0`), les fonctions `set_params` et `get_params` sont vides . Le backward de cette couche est :
$$ g_e[i]=\phi'(X[i])g_s[i] \quad \forall 1\le i\le p$$
où $g_e$ et $g_s$ sont respectivement le gradient d'entrée et le gradient de sortie. On rappelle que la variable `X` a été sauvegardée dans la variable `self.save_X`.

Implémentez cette couche et testez le code suivant, pour le gradient local, ou pour `get_params` on rendra la valeur `None`.

In [5]:
import Neural as Neur
L=Neur.Arctan()
np.random.seed(10)
X=np.random.randn(4,2)
grad_sortie=np.random.randn(4,2)
print('nb_params=',L.nb_params)
print('forward=',L.forward(X))
print('backward=',L.backward(grad_sortie))
# Vous devez trouver
#nb_params= 0
#forward= [[ 0.92666583  0.62090688]
# [-0.99647548 -0.00838365]
# [ 0.55596017 -0.6240794 ]
# [ 0.25952369  0.10812518]]
#backward=  (None, array([[ 0.00154751, -0.11550505],
#       [ 0.12780186,  1.20295282],
#       [-0.69626624,  0.67715401],
#       [ 0.21357394,  0.43995373]]))


TypeError: __init__() missing 2 required positional arguments: 'nb_entree' and 'nb_sortie'

# D'autres couches
Sur le modèle de la couche Arctan, on peut construire un certain nombre de couches en modifiant la fonction $\Phi$ (et évidemment la fonction $\Phi'$). Ce sont toutes des couches simples sans paramètres et qui ne font qu'appliquer une non-linéarité aux données. Voici un tableau de quelques couches utilisées, de leur $\Phi$ et $\Phi'$ correspondant :


| Nom            |     $\Phi(X)$        |        $\Phi'(X)$                 |
| :------------  | :---------------:    | ----------------------:           |
| Sigmoïde       | $$\frac 1 {1+e^{-X}}$$ | $$\frac {e^{-X}} {(1+e^{-X})^2}$$ |
| RELU           |   $$\max(X,0)$$        |   $$\max(\frac{X}{|X|},0)$$          |
| ABS            |     $$|X|$$            |        $$\frac{X}{|X|}$$              |

# La structure des données
Avant de continuer, il nous faut dire quelle est la structure des données.
On suppose que l'on a $n$ données différentes dans $\mathbb{R}^p$. Ces données sont stockées dans une grande matrice de taille $(p,n)$ dont la $j$-ème colonne est un vecteur de taille $\mathbb{R}^p$ qui représente la $j$-eme donnée.
On note cette matrice $X_j[i]$ avec $1\le i \le p$ et $1\le j \le n$ tel que le vecteur $X_j$ est la $j$-eme donnée d'entrée.
Ainsi l'exemple suivant représente 4 données dans $\mathbb{R}^3$

In [6]:
X =np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
print(X)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


La première opération que l'on veut faire est, étant donné une matrice $A$ de taille $(q,p)$, de trouver la matrice $Y$ de taille $(q,n)$ telle que pour chaque donnée $j$, on ait $Y_j=AX_j$. Dans l'exemple suivant $q=2$.

In [7]:
A=np.array([[1,2,3],[4,5,6]])
print('A=',A)
Y=A.dot(X)
print ('Y=',Y)
# Vous devez trouver
#A= [[1 2 3]
# [4 5 6]]
#Y= [[ 38  44  50  56]
# [ 83  98 113 128]]

A= [[1 2 3]
 [4 5 6]]
Y= [[ 38  44  50  56]
 [ 83  98 113 128]]


Maintenant on veut ajouter à $Y$ un vecteur $b \in \mathbb{R}^q$ tel que pour chaque donnée $j$ on ait $Z_j=Y_j+b$. Pour cela on utilise la commande suivante

In [3]:
b=np.array([1,2])
print(np.outer(b,np.ones(5)))

[[1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]]


 Vous devez comprendre la commande précédente et vous en servir pour calculer le vecteur $Z$ tel que $Z_j=AX_j+b$ dans l'exemple suivant :

In [9]:
X =np.array([[1,1,3,4],[5,6,7,8],[9,10,11,12]])
A=np.array([[1,5,3],[4,5,6]])
b=np.array([1,3])
# Remplir ici!
Z=A.dot(X)+np.outer(b,np.ones(4))
print(Z)
# Vous devez trouver
#[[ 54.  62.  72.  81.]
# [ 86.  97. 116. 131.]]

[[ 54.  62.  72.  81.]
 [ 86.  97. 116. 131.]]


# Implémentation de la couche Dense
Une couche "dense" est une couche qui prend $X$ un vecteur de taille $p$ et rend $Y$ un vecteur de taille $q$ tel que
$$Y=AX+b,$$
Où $A$ est une matrice et $b$ un vecteur de taille $q$. La matrice $A$ et le vecteur $b$ sont des paramètres de la couche. 
Nous allons dans un premier temps s'intéresser uniquement à la fonction `__init__`.

La fonction `__init__` prend en argument deux entiers `nb_entree` et `nb_sortie` (notés $p$ et $q$ ici) correspondant à respectivement à la taille des vecteurs d'entrée et la taille des vecteurs de sortie. Ces nombres doivent être stockés dans les variables internes `self.n_entree` et `self.n_sortie`. De plus la fonction `__init__` va tirer de manière aléatoire une matrice $A$ de taille $(q,p)$ (stockée dans `self.A`) et un vecteur $b$ (stocké dans `self.b`) de taille $q$. On utilisera un tirage selon une normale $(0,1)$ avec la fonction `random.randn` pour cela.

In [10]:
# TEST de la classe
np.random.seed(10)
import Neural as Neur
L=Neur.Dense(3,2)
print(L.n_entree)
print(L.n_sortie)
print(L.nb_params)
print('A=',L.A)
print('b=',L.b)
# Vous devez trouver
#3
#2
#8
#A=[[ 1.3315865   0.71527897 -1.54540029]
#[-0.00838385  0.62133597 -0.72008556]]
#b=[ 0.26551159  0.10854853]

3
2
8
A= [[ 1.3315865   0.71527897 -1.54540029]
 [-0.00838385  0.62133597 -0.72008556]]
b= [0.26551159 0.10854853]


On remplit maintenant la fonction `forward` de la classe `Layer`. Etant donné une matrice $X$ de taille $(p,n)$ (où $n$ représente le nombre de données), la fonction `forward` calcule $Y$ de taille $(q,n)$ telle que $Y_j=AX_j+b$ pour tout $j$ entre $1$ et $n$ (on rappelle que $n$ est donné par `X.shape[1]`). On utilisera la section précédente pour faire ce calcul, on fera notamment attention à ne pas utiliser le vecteur $b$ directement dans la somme.

In [11]:
np.random.seed(10)
import Neural as Neur
L = Neur.Dense(3,2)
X =np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
print(L.forward(X))
# Vous devez trouver
#[[-8.73510967 -8.23364448 -7.73217929 -7.23071411]
# [-3.2739255  -3.38105894 -3.48819237 -3.59532581]]

[[-8.73510967 -8.23364448 -7.73217929 -7.23071411]
 [-3.2739255  -3.38105894 -3.48819237 -3.59532581]]


Nous allons maintenant nous intéresser aux fonctions `get_params` et `set_params`. La fonction `get_params` rend dans un grand vecteur les coefficients de la matrice $A$ suivis des coefficients du vecteur $b$. Pour récupérer les coefficients d'une matrice sous forme d'un grand tableau, il existe la fonction `ravel` sous python. La fonction `set_params` prend en argument un grand vecteur et remplit tout d'abord les coefficients de la matrice $A$ puis les coefficients du vecteur $b$ avec les coefficients du grand vecteur. Pour recréer, la matrice $A$, on utilisera la fonction `reshape` de numpy.

In [12]:
np.random.seed(10)
import Neural as Neur
L = Neur.Dense(3,2)
print('params=',L.get_params())
theta=np.random.randn(8)
L.set_params(theta)
print('theta=',theta)
print('A=',L.A)
print('b=',L.b)
#Vous devez trouver
#params= [ 1.3315865   0.71527897 -1.54540029 -0.00838385  0.62133597 -0.72008556
#  0.26551159  0.10854853]
#theta= [ 0.00429143 -0.17460021  0.43302619  1.20303737 -0.96506567  1.02827408
#  0.22863013  0.44513761]
#A= [[ 0.00429143 -0.17460021  0.43302619]
# [ 1.20303737 -0.96506567  1.02827408]]
#b= [0.22863013 0.44513761]

params= [ 1.3315865   0.71527897 -1.54540029 -0.00838385  0.62133597 -0.72008556
  0.26551159  0.10854853]
theta= [ 0.00429143 -0.17460021  0.43302619  1.20303737 -0.96506567  1.02827408
  0.22863013  0.44513761]
A= [[ 0.00429143 -0.17460021  0.43302619]
 [ 1.20303737 -0.96506567  1.02827408]]
b= [0.22863013 0.44513761]


 C'était simple... Maintenant il faut coder la rétropropagation pour la couche dense. On rappelle que si la couche est de taille $(p,q)$ avec $n$ données alors la fonction `backward` prend en entrée `grad_sortie` (noté $g_s$ ici) un vecteur de taille $(q,n)$, calcule le gradient par rapport à $A$ et $b$ (qui sont donc des vecteurs de taille $(q,p)$ et $q$ respectivement et qui sont notés $g_A$ et $g_b$) le transforme en un vecteur de taille `self.nb_params` (comme la fonction `get_params`) et rend un vecteur `grad_entree` de taille $(p,n)$ (noté $g_e$ )qui sert à rétropropager le gradient aux couches précédentes. Les formules de calcul sont :

$$g_e = A^T g_s$$   
$$g_A = g_s X^T$$
$$g_b[i] = \sum_j g_s[i,j]$$
Implémentez la fonction backward de Layer_dense et testez votre code ci-dessous :

In [13]:
np.random.seed(10)
import Neural as Neur
L = Neur.Dense(3,2)
X=np.random.randn(3,4)
grad_sortie=np.random.randn(2,4)
L.forward(X)
gl,ge=L.backward(grad_sortie)
print('grad_local=',gl)
print('grad_entree=',ge)
#Vous devez trouver
#grad_local= [ 3.28032607  1.23844345 -0.16801192  1.43755815  1.28044748 -2.41352966
# -1.07006308  4.29345906]
#grad_entree= [[-2.64293715 -2.33547403  0.35346419  3.16406972]
# [-0.71643766 -0.2077372   0.25191937  2.57454243]
# [ 2.24722802  1.48977695 -0.48258083 -4.69240621]]

grad_local= [ 3.28032607  1.23844345 -0.16801192  1.43755815  1.28044748 -2.41352966
 -1.07006308  4.29345906]
grad_entree= [[-2.64293715 -2.33547403  0.35346419  3.16406972]
 [-0.71643766 -0.2077372   0.25191937  2.57454243]
 [ 2.24722802  1.48977695 -0.48258083 -4.69240621]]


## Construction de la couche de perte L2
Nous allons maintenant construire une classe qui correspond à la couche de perte $L_2$. Cette couche n'a pas de paramètres, elle a des données $D$ stockées et le forward consiste à calculer 
$$Y=\frac{1}{2}\Vert X-D\Vert^2.$$
La variable $Y$ est un réel et c'est classiquement la dernière couche du réseau de Neurone, la couche qui nous permet de mesurer l'écart entre $X$ et les données $D$.

Pour le backward, cette couche n'a pas besoin de gradient de sortie, elle n'a pas de gradient local et son gradient d'entrée est $X-D$. Implémentez la couche de perte $L_2$ dans une classe nommée `Loss_L2` et testez le code ci-dessous :


In [31]:
np.random.seed(10)
import Neural as Neur
D=np.random.randn(3,2)
X=np.random.randn(3,2)
L = Neur.Loss_L2(D)
print(L.nb_params)
print(L.forward(X))
print(L.backward(None))
#Vous devez trouver
#0
#3.8338361400772234
#(None, array([[-1.06607492, -0.60673045],
#       [ 1.54969172, -0.16621636],
#       [-0.18830978,  1.92312293]]))


0
3.8338361400772234
(None, array([[-1.06607492, -0.60673045],
       [ 1.54969172, -0.16621636],
       [-0.18830978,  1.92312293]]))


# Construction du réseau de Neurones
Nous allons maintenant nous intéresser à la construction du réseau de Neurone en lui même. Un réseau de Neurone est essentiellement une liste de couches  que l'on exécute successivement.

Pour des raisons pratiques, la classe Network a exactement les mêmes noms de variables que la classe Layer, en fait on pourrait (et c'est même quelquefois très intéressant) considérer les réseaux de neurones comme une couche et faire plusieurs couches de réseau de neurone. La classe d'un réseau de neurone se nommera `Network`

On s'intéresse d'abord aux fonctions `__init__` et aux fonctions `set_params` et `get_params`. La fonction `__init__` prend en argument la variable `list_layers` qui est une liste de couches. Cette liste doit être sauvée dans une variable `self.list_layers`.

La variable `self.nb_params` doit être calculée. Par définition, nous supposerons que `self.nb_params` est juste la somme des tailles du vecteur des paramètres de chaque couche. Il faut donc parcourir la liste des couches pour calculer cette somme.

Ensuite il faut remplir les fonctions `set_params` et `get_params`, la fonction `set_params` prend en argument un vecteur de taille `self.nb_params` et affecte les premiers coefficients du vecteur à la première couche, puis les suivants à la deuxième couche, etc.. La fonction `get_params` lance successivement les fonctions `get_params` des couches de la liste `self.list_layers` et stocke tous les résultats dans un grand tableau.

In [37]:
np.random.seed(10)
import Neural as Neur
D=np.random.randn(4,10)
X=np.random.randn(3,10)

L1=Neur.Dense(3,2)
L2=Neur.Arctan()
L3=Neur.Dense(2,6)
L4=Neur.Arctan()
L5=Neur.Dense(6,4)
L6=Neur.Loss_L2(D)


N=Neur.Network([L1,L2,L3,L4,L5,L6])
           
print(N.nb_params)
print('params=',N.get_params())
theta=np.random.randn(N.nb_params)
N.set_params(theta)
print(np.linalg.norm(N.get_params()-theta))
# Vous devez obtenir
#54
#params= [ 0.31935642  0.4609029  -0.21578989  0.98907246  0.31475378  2.46765106
# -1.50832149  0.62060066 -1.04513254 -0.79800882  1.98508459  1.74481415
# -1.85618548 -0.2227737  -0.06584785 -2.13171211 -0.04883051  0.39334122
#  0.21726515 -1.99439377  1.10770823  0.24454398 -0.06191203 -0.75389296
#  0.71195902  0.91826915 -0.48209314  0.08958761  0.82699862 -1.95451212
#  0.11747566 -1.90745689 -0.92290926  0.46975143 -0.14436676 -0.40013835
# -0.29598385  0.84820861  0.70683045 -0.78726893  0.29294072 -0.47080725
#  2.40432561 -0.73935674 -0.31282876 -0.34888192 -0.43902624  0.14110417
#  0.27304932 -1.61857075 -0.57311336 -1.32044755  1.23620533  2.46532508]
#0.0

54
params= [ 0.31935642  0.4609029  -0.21578989  0.98907246  0.31475378  2.46765106
 -1.50832149  0.62060066 -1.04513254 -0.79800882  1.98508459  1.74481415
 -1.85618548 -0.2227737  -0.06584785 -2.13171211 -0.04883051  0.39334122
  0.21726515 -1.99439377  1.10770823  0.24454398 -0.06191203 -0.75389296
  0.71195902  0.91826915 -0.48209314  0.08958761  0.82699862 -1.95451212
  0.11747566 -1.90745689 -0.92290926  0.46975143 -0.14436676 -0.40013835
 -0.29598385  0.84820861  0.70683045 -0.78726893  0.29294072 -0.47080725
  2.40432561 -0.73935674 -0.31282876 -0.34888192 -0.43902624  0.14110417
  0.27304932 -1.61857075 -0.57311336 -1.32044755  1.23620533  2.46532508]
0.0


On va programmer le `forward` et le `backward` du réseau de Neurone. 
Pour le `forward`, il s'agit juste de prendre la variable `X`, de la copier dans une variable `Z` et de faire passer cette variable `Z` dans chaque couche de la liste. On rend le résultat final. Il est important de copier la variable `X` dans une variable temporaire, sinon notre variable `X` sera modifiée par l'algorithme !!
Pour le backward, il faut juste faire passer le gradient dans la liste à l'envers. Pour parcourir la liste à l'envers, il faut utiliser la fonction `reverse` de python. Tous les gradients locaux doivent être sauvés dans un grand vecteur, en utilisant un code similaire à la fonction `get_params`.

In [40]:
np.random.seed(10)
import Neural as Neur
D=np.random.randn(4,10)
X=np.random.randn(3,10)

L1=Neur.Dense(3,2)
L2=Neur.Arctan()
L3=Neur.Dense(2,6)
L4=Neur.Arctan()
L5=Neur.Dense(6,4)
L6=Neur.Loss_L2(D)


N=Neur.Network([L1,L2,L3,L4,L5,L6])

a=N.forward(X)
b,c=N.backward(None)
print('a=',a)
print('b=',b)
print('c=',c)
# Vous devez trouver
#a= 221.22208566711836
#b= [ -0.98368166 -13.25213968  -6.43358648  21.26910607 -14.87907661
#  -0.35630809 -23.31164944  27.82326981  -8.54800412   0.60620022
#  19.75002508 -12.54967804  -2.11398405   7.88699083 -10.23572316
# -16.12049842 -55.35679888   5.89818847  35.3167564  -23.76380918
#   9.58230795 -20.60100053   2.13470619  12.16682271  60.20318534
# -37.40341197 -11.30231307  24.605031    -7.02968989 -29.58355645
#   3.70228854 -32.28218831 -30.89848629  24.74850639 -28.1221249
#   2.2068016  -15.89016799 -14.56113369  39.54725178 -21.85809805
#  37.99499607 -23.31829384  26.61064478   1.60179509  13.52040638
#  -4.05726552  13.42191519 -10.62904716  10.35644233  -3.94820181
#  -5.71322575 -27.67476451  38.3148709   13.97693659]
#c= [[ 9.62116626e+00 -4.87640076e+00  1.06443747e-01 -3.06919940e+00
#  -3.59066812e+00 -5.20284380e+00 -1.51608936e+00 -7.42926556e-01
#   1.58659223e+01  1.34791006e+01]
# [ 3.21815414e-01 -1.85582037e+00 -8.85800384e-01 -9.69444862e-01
#  -1.57117837e+00 -1.78510096e+00 -2.74246971e+00  1.78176598e-02
#   3.64128666e+00  3.84196736e+00]
# [ 3.17260799e+01 -1.13094336e+01  2.85752584e+00 -7.67787992e+00
#  -7.75070537e+00 -1.26159745e+01  2.58695138e+00 -2.57007417e+00
#   4.35516287e+01  3.48904212e+01]]


a= 221.22208566711836
b= [ -0.98368166 -13.25213968  -6.43358648  21.26910607 -14.87907661
  -0.35630809 -23.31164944  27.82326981  -8.54800412   0.60620022
  19.75002508 -12.54967804  -2.11398405   7.88699083 -10.23572316
 -16.12049842 -55.35679888   5.89818847  35.3167564  -23.76380918
   9.58230795 -20.60100053   2.13470619  12.16682271  60.20318534
 -37.40341197 -11.30231307  24.605031    -7.02968989 -29.58355645
   3.70228854 -32.28218831 -30.89848629  24.74850639 -28.1221249
   2.2068016  -15.89016799 -14.56113369  39.54725178 -21.85809805
  37.99499607 -23.31829384  26.61064478   1.60179509  13.52040638
  -4.05726552  13.42191519 -10.62904716  10.35644233  -3.94820181
  -5.71322575 -27.67476451  38.3148709   13.97693659]
c= [[ 9.62116626e+00 -4.87640076e+00  1.06443747e-01 -3.06919940e+00
  -3.59066812e+00 -5.20284380e+00 -1.51608936e+00 -7.42926556e-01
   1.58659223e+01  1.34791006e+01]
 [ 3.21815414e-01 -1.85582037e+00 -8.85800384e-01 -9.69444862e-01
  -1.57117837e+00 -1.78510

Et voilà c'est fini... Vous êtes fin prêts à faire des réseaux de neurone maintenant.